In [4]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%95%9C%EA%B0%95%EC%88%98/place/13483118?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            try:
                # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[5]/a[1]'
                review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[4]/a'
                review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                # 리뷰 텍스트와 날짜를 사진 여부에 따라 가져오기
                try:
                    # 사진이 있을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                except NoSuchElementException:
                    # 사진이 없을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_text_element)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_date_element)

                review_text = review_text_element.text.strip()
                review_date = review_date_element.text.strip()

                if review_text:
                    reviews.append((review_date, review_text))
                    review_count += 1
                    print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                else:
                    print(f"리뷰 {review_count}: 내용 없음")
                    review_count += 1

            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음")
                continue

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


가게 이름: 한강수
업종 카테고리: 정육식당
별점: 4.21
방문자 리뷰: 174
블로그 리뷰: 50
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 105
리뷰 1: 동네 친구들이랑 한잔하러 왔어요
주차하기 편하고 고기 질 좋고 가성비 좋아요😊😊 (2024년 10월 8일 화요일)

리뷰 2: 강남 뱅뱅매장 뒤편에 위치해있습미당~
고기집으로 퀄리티가 좋고 맛도 좋아용
점심메뉴 회덮밥 최애 메뉴네욯ㅎ
양도 많아서 아주 푸짐~
점심 시간대 피해가야 줄을 서지 않을수 있음! (2024년 7월 31일 수요일)

리뷰 3: 모두 친철하시고, 고기질 뿐만 아니라 반찬하나하나까지 너무 맛있어요~👍🏻 (2024년 10월 12일 토요일)

리뷰 4: 굿 (2024년 10월 29일 화요일)

리뷰 5: 굿 (2024년 9월 6일 금요일)

리뷰 6: 맛있게 먹었습니다~~~점심때도 손님이 많아요ㅋ (2024년 7월 3일 수요일)

리뷰 7: 건강한 점심 맛집
한우 맛집
점심 육회비빔밥
가성비 좋아요 (2024년 4월 5일 금요일)

리뷰 8: 꺄~~~~ 철수 전에 소고기 개이득
밑반찬도 된장찌개도 짱마싯고 한우도 너무 맛있고 사장님도 친절하십니다,, (2024년 3월 12일 화요일)

리뷰 9: 굿굿 (2024년 10월 4일 금요일)

리뷰 10: 반찬이 잘 나온다. 음식 나올때까지 대기가 좀 있는 편. 손님들이 많다. 직원분들은 친절하진 않다. 간이 쎈 편이다. 육회는 신선해서 맛있었다. (2024년 4월 12일 금요일)

하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
리뷰 11: 굿굿 (2024년 7월 19일 금요일)

리뷰 12: 굿굿 (2024년 7월 5일 금요일)

리뷰 13: 강남과 양재사이 숨겨진? 가성비 맛집 (2024년 4월 26일 금요일)

리뷰 14: 굿 (2024년 6월 

In [3]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/entry/place/1182000003?lng=127.0366824&lat=37.4937583&placePath=%2Fhome&entry=plt&searchType=place&c=15.00,0,0,0,dh')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            try:
                # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[5]/a[1]'
                review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[4]/a'
                review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                # 리뷰 텍스트와 날짜를 사진 여부에 따라 가져오기
                try:
                    # 사진이 있을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                except NoSuchElementException:
                    # 사진이 없을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_text_element)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_date_element)

                review_text = review_text_element.text.strip()
                review_date = review_date_element.text.strip()

                if review_text:
                    reviews.append((review_date, review_text))
                    review_count += 1
                    print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                else:
                    print(f"리뷰 {review_count}: 내용 없음")
                    review_count += 1

            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음")
                continue

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x009DEBD3+24307]
	(No symbol) [0x00968D74]
	(No symbol) [0x0084C323]
	(No symbol) [0x0082E00B]
	(No symbol) [0x008B5F6F]
	(No symbol) [0x008C8EC9]
	(No symbol) [0x008AFC26]
	(No symbol) [0x0088219C]
	(No symbol) [0x0088311D]
	GetHandleVerifier [0x00C88D93+2818227]
	GetHandleVerifier [0x00CE542E+3196750]
	GetHandleVerifier [0x00CDD9D2+3165426]
	GetHandleVerifier [0x00A7DA70+675216]
	(No symbol) [0x00971B3D]
	(No symbol) [0x0096EA18]
	(No symbol) [0x0096EBB5]
	(No symbol) [0x00961640]
	BaseThreadInitThunk [0x763FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7743809E+286]
	RtlGetAppContainerNamedObjectPath [0x7743806E+238]

